In [1]:
import ee
import google.auth
import math
import numpy as np
import tensorflow as tf
import folium
import time
import geopandas
import pandas


In [2]:
# this is needed to Successfully save authorization token. from ee.Authenticate()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
ee.Authenticate()


Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
# bands used for prediction
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'] # bands with <= 30 resolution

COUNTRY_GEOMETRY = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', 'Ukraine'))
COUNTRY_LATLON = 50., 31 # coordinate of center of ukraine

## Reading Labeled Shape File

In [6]:
shapeFile = geopandas.read_file("../data/validation_data/merged_harvest_validation_20220919.shp")
shapeFile.head()

,fid,cat,id,x,y,lat,lon,strata,set,rd_id,lab_set1,val_set1,com_set1,lab_set2,val_set2,com_set2,finHarvDat,geometry
0,249.0,249.0,248.0,-321360.0,639156.0,50.655048,25.458684,20.0,free_ukraine,51.0,Shabri,1.0,Harvested 11/08/22,Josef,1.0,Harvested 05/08-11/08/22,11/08/22,POINT (2834056.683 6560486.924)
1,165.0,165.0,164.0,447948.0,607452.0,50.294633,36.289471,20.0,free_ukraine,16.0,Shabri,0.0,Spring,Josef,0.0,NaN,NaN,POINT (4039734.283 6497444.336)
2,400.0,400.0,399.0,179688.0,655860.0,50.860849,32.548484,20.0,free_ukraine,32.0,Shabri,2.0,"Unclear, keeps greening up and browing down",Fangjie,1.0,Harvested 31/07/2022 and 29/08/2022,NaN,POINT (3623289.622 6596702.279)
3,387.0,387.0,386.0,115320.0,661740.0,50.928689,31.637558,20.0,free_ukraine,69.0,Shabri,0.0,Interesting,Fangjie,1.0,Harvested 17/07/2022,NaN,POINT (3521885.751 6608675.134)
4,306.0,306.0,305.0,543120.0,393504.0,48.307964,37.348245,20.0,free_ukraine,8.0,Shabri,1.0,Harvested 23/07/22,Blake,0.0,NaN,23/07/22,POINT (4157596.478 6158229.471)


In [7]:
points_from_sph_df = shapeFile[['lat', 'lon', 'val_set1', 'finHarvDat']].dropna(subset=['lat', 'lon', 'val_set1'])
points_from_sph_df['is_harvested'] = points_from_sph_df['val_set1'].apply(lambda x: x == 1)
points_from_sph_df = points_from_sph_df.drop(['val_set1'], axis=1)
points_from_sph_df.finHarvDat = points_from_sph_df.finHarvDat.apply(lambda x: str(x))
points_from_sph_df['point_id'] = np.arange(0, points_from_sph_df.shape[0], 1, dtype=int)
print(points_from_sph_df.head(), points_from_sph_df.shape)

         lat        lon finHarvDat  is_harvested  point_id
0  50.655048  25.458684   11/08/22          True         0
1  50.294633  36.289471        nan         False         1
2  50.860849  32.548484        nan         False         2
3  50.928689  31.637558        nan         False         3
4  48.307964  37.348245   23/07/22          True         4 (558, 5)


In [8]:
def overlay_points(img: ee.Image, df:pandas.DataFrame, coordinate_col_names:(str, str)=('lon', 'lat')) -> ee.FeatureCollection:
  """
  Overlays the points at df onto img. then, creates a table with the overlayed points(saved as ee.FeatureCollection).
  The ee.FeatureCollection is exported into Drive in a GeoJson format.
  So, the bands reflectances from img are put into the dataset described as the dataframe(df), then exported as .GeoJson.
  
  Args:
      img (ee.Image): _description_
      df (pandas.DataFrame): cointains longitude and latitude cols, describing the points to be overlayed.
      coordinate_col_names (str, str, optional): the name of columns (in the dataframe df) that have the coordinates. Defaults to ('lon', 'lat').

  Returns:
      ee.FeatureCollection: _description_
      
  Usage:
      export_to_drive(overlay_points(image, points_from_sph_df), 'points_from_sph')
      # will save points_from_sph.geojson into drive.
  """
  # Convert pandas dataframe to an ee.FeatureCollection
  def createFeature(row):
      lon = coordinate_col_names[0]
      lat = coordinate_col_names[1]
      geometry = ee.Geometry.Point([row[lon], row[lat]])
      #print(df.columns.values)
      dic = {}
      for col_name in df.columns.values:
        dic[col_name] = row[col_name]
        
      return ee.Feature(geometry, dic)

  features = points_from_sph_df.apply(createFeature, axis=1).tolist()
  fc = ee.FeatureCollection(features)


  # Overlay the points on the imagery to get training.
  overlayed_fc = img.sampleRegions(
    collection= fc,
    scale= 10 # maybe we should make this 10 instead
  )
  return overlayed_fc


def export_to_drive(fc: ee.FeatureCollection, file:str, folder:str="NASA_Harvest"):

  # Export the ee.FeatureCollection as a .GeoJSON file.
  task = ee.batch.Export.table.toDrive(**{
    'collection': fc,
    'description':file,
    'fileFormat': 'GeoJSON',
    'folder': folder
  })
  task.start()


  print('----')
  print(f'Polling for file name= {file}...')
  while task.active():
    time.sleep(5)
  print(f'Wrote {file}.GeoJSON. Check {folder} folder in Drive.')
  print('----')

In [9]:
from datetime import datetime

def read_geojson(file:str, date_col_name='finHarvDat') -> pandas.DataFrame:
    """
        Expects the date column to have this format "dd/mm/yy", example: "14/07/22".
        Expects that the file path looks like this: ../data/{file}.geojson
    Args:
        file (str): name of file (without the .geojson extention)
        date_col_name (str, optional):
        if df has no date column, pass None. Defaults to 'finHarvDat'.

    Returns:
        pandas.DataFrame
        
    Usage:
            read_geojson(file='points_from_sph').head()
    """

    def to_datetime(string):
        if(string == 'nan' or not string):
            return None
        splitted = string.split('/')
        day = int(splitted[0])
        month = int(splitted[1])
        year = 2000 + int(splitted[2])
        datetime_object = datetime(year=year, month = month, day=day)#datetime.strptime(string, '%d-%m-%Y')
        return datetime_object
    
    df = geopandas.read_file(f"../data/{file}.geojson")
    
    if(date_col_name != None):
        df[date_col_name] = df[date_col_name].apply(to_datetime)
        
    return df

## Create 3-week image collection from sentindel2 in 2022

reference: https://medium.com/@moraesd90/creating-monthly-ndvi-composites-sentinel-2-on-google-earth-engine-a5c2d49bc9ca

In [33]:
def get_image(s2_img_collection: ee.ImageCollection) -> ee.Image:
    """
    Returns:
        ee.image: a cloud masked sentinel-2 image.
    """
    def cloudmask_and_clip(image: ee.Image) -> ee.Image:
        opaqueClouds_mask = 1 << 10
        cirrusClouds_mask =1 << 11
        bit_mask =opaqueClouds_mask | cirrusClouds_mask
        qa = image.select('QA60')
        mask = qa.bitwiseAnd(bit_mask).eq(0)
        return image.clip(COUNTRY_GEOMETRY).updateMask(mask)
    
    def add_ndvi(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        image = image.addBands(ndvi.toFloat())
        return image.toFloat()
    
    default_value = 0.0
    image = s2_img_collection.map(cloudmask_and_clip).select(BANDS).filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)).median().unmask(default_value).float()
    image = add_ndvi(image)
    return image

In [34]:
DATE_START = ee.Date('2022-01-01')
DATE_END= ee.Date('2022-12-27')
SURF_REF_SEN2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(DATE_START, DATE_END)

# start_weeks.getInfo -> [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52]
total_weeks = ee.Number(DATE_END.difference(DATE_START, 'week')).round().getInfo()
start_weeks = ee.List.sequence(1, total_weeks, 3)

def extract_subset(start_week):
    
    start = DATE_START.advance(start_week, 'week')
    end = start.advance(3, 'week').advance(-1, 'day')
    
    def getCollection():
        return SURF_REF_SEN2.filterDate(start, end)
    
    img_collection = getCollection()
    return get_image(img_collection)
    
    

# Map the extract_subset function over the list of start weeks to create a new image collection that contains the subsets of the original image collection
new_img_collection = ee.ImageCollection.fromImages(start_weeks.map(extract_subset))
num_of_images = new_img_collection.size().getInfo()
# Print the number of images in the new image collection
print('Number of images in the new image collection:', num_of_images)
images_list = new_img_collection.toList(num_of_images)

Number of images in the new image collection: 17


In [35]:
images_list

In [1]:
image = ee.Image(images_list.get(6))

NameError: name 'ee' is not defined

In [37]:
vis_params = {
  "min": 0,
  "max": 3000,
  "bands": ["B4", "B3", "B2"],
}
map = folium.Map(location=COUNTRY_LATLON, zoom_start=13)
image=image.clip(COUNTRY_GEOMETRY)
mapid = image.getMapId(vis_params)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
folium.LayerControl().add_to(map)


In [38]:
map

In [13]:
start_end_dates = []
start_weeks = ee.List.sequence(1, total_weeks, 3)

# record the dates into start_end_dates array
def eeDate_to_datetime(eeDate: ee.Date)->datetime:
    year = eeDate.get('year').getInfo()
    month = eeDate.get('month').getInfo()
    day = eeDate.get('day').getInfo()
    return datetime(year=year, month=month, day=day)

for idx in range(num_of_images):
    start_week = start_weeks.get(idx).getInfo()
    start = DATE_START.advance(start_week, 'week')
    end = start.advance(3, 'week').advance(-1, 'day')
    start_end_dates.append((eeDate_to_datetime(start), eeDate_to_datetime(end)))

In [14]:
start_end_dates[0], len(start_end_dates)

((datetime.datetime(2022, 1, 8, 0, 0), datetime.datetime(2022, 1, 28, 0, 0)),
 17)

In [15]:
file_names = [f'img{idx}_overlayed' for idx in range(num_of_images)]

In [16]:
%%script echo skipping

# for each image, overylay the points(aka export to drive)
for idx in range(num_of_images):
    currImg = ee.Image(images_list.get(idx))
    file = f'img{idx}_overlayed'
    export_to_drive(overlay_points(currImg, points_from_sph_df),file=file )    

Couldn't find program: 'echo'


## Download From Drive & Move to 'data' Folder.

In [17]:

# true if (is_harvested is true) and if (the finHarvDat is within start_date & end_date)
def get__is_within_period(row):
    def date_within_range(dateToCheck:datetime, startDate:datetime, endDate:datetime):
        """credit: https://stackoverflow.com/users/22656/jon-skeet"""
        return dateToCheck >= startDate and dateToCheck <= endDate
    return row['is_harvested'] and date_within_range(row['finHarvDat'], row['start_date'], row['end_date'])
    
#samples = ee.List([]) # containing images
samples = [None] * (num_of_images) # sample[0] is None

path_inside_data_folder='overlayed_3week_images/'

for idx in range(num_of_images):
    curr_img_df = read_geojson(file=path_inside_data_folder + file_names[idx])
    curr_img_df = curr_img_df.sort_values('point_id') # to make sure the points are aligned when we subtract ndvi values below
    
    # add time cols
    start, end = start_end_dates[idx]
    curr_img_df['start_date'] = np.tile(np.array([start]), curr_img_df.shape[0])
    curr_img_df['end_date'] = np.tile(np.array([end]), curr_img_df.shape[0])
    
    curr_img_df['is_within_period'] = curr_img_df.apply(get__is_within_period, axis=1)
    
    curr_img_df['image_idx'] = np.tile(np.array(['i'+str(idx)]), curr_img_df.shape[0])
    curr_img_df.rename(columns = {'point_id':'point_idx'}, inplace = True)
    curr_img_df.point_idx = curr_img_df.point_idx.apply(lambda x: 'p' + str(x))
    
    samples[idx] = curr_img_df

In [18]:
merged_images = pandas.concat(samples, sort=False)
merged_images = merged_images.drop(['geometry', 'id', 'is_harvested'], axis=1)
merged_images[BANDS] /= 10000 # # divide by 10000 bc the bands are scaled by 10000
                                #(according to https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2)
merged_images.shape

(9486, 19)

In [19]:
merged_images.head()

,B11,B12,B2,B3,B4,B5,B6,B7,B8,B8A,NDVI,finHarvDat,lat,lon,point_idx,start_date,end_date,is_within_period,image_idx
0,0.1000,0.1000,0.10,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.000000,2022-08-11,50.655048,25.458684,p0,2022-01-08,2022-01-28,False,i0
1,0.1000,0.1000,0.10,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.000000,NaT,50.294633,36.289471,p1,2022-01-08,2022-01-28,False,i0
2,0.1000,0.1000,0.10,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.000000,NaT,50.860849,32.548484,p2,2022-01-08,2022-01-28,False,i0
3,0.1000,0.1000,0.10,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.000000,NaT,50.928689,31.637558,p3,2022-01-08,2022-01-28,False,i0
4,0.1424,0.1083,0.03,0.0432,0.0631,0.0715,0.0776,0.0851,0.1066,0.1043,0.256335,2022-07-23,48.307964,37.348245,p4,2022-01-08,2022-01-28,False,i0


In [20]:
sum(merged_images.is_within_period) # we expect 366

366

In [21]:
# convert pandas df to geopandas df
merged_samples_gdf = geopandas.GeoDataFrame(
    merged_images, geometry=geopandas.points_from_xy(merged_images.lon, merged_images.lat))

# save dataset
merged_samples_gdf.to_file('../data/merged_images.geojson', driver='GeoJSON')